# IMPORTANT 
Make sure you've imported the [environment.yml](https://github.com/Snowflake-Labs/sfguide-intro-to-machine-learning-with-snowflake-ml-for-python/blob/main/notebooks/environment.yml) file provided in the git repo on the left sidebar.

This will ensure if you have the right packages needed to run this Notebook.

## 1. Data Ingestion

The `diamonds` dataset has been widely used in data science and machine learning. We will use it to demonstrate Snowflake's native data science transformers in terms of database functionality and Spark & Pandas comportablity, using non-synthetic and statistically appropriate data that is well known to the ML community.



### Import Libraries

In [ ]:
# Snowpark for Python
#from snowflake.snowpark.types import DoubleType
#from snowflake.snowpark.types import DecimalType

import snowflake.snowpark.functions as F
import streamlit as st

# Snowflake Cortex (Generative AI)
from snowflake.cortex import complete


# Snowflake ML
import snowflake.ml.modeling.preprocessing as snowml
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.metrics.correlation import correlation
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.registry import Registry
from snowflake.ml._internal.utils import identifier
from snowflake.ml.modeling.metrics import mean_absolute_percentage_error


# OSS data analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Misc
#import json
#import joblib

# warning suppresion
import warnings; warnings.simplefilter('ignore')

### Setup context just to be sure (Compute, DB, Schema)

In [ ]:
-- Using Warehouse, Database, and Schema created during Setup
USE WAREHOUSE ML_HOL_WH;
USE DATABASE ML_HOL_DB;
USE SCHEMA ML_HOL_SCHEMA;

In [ ]:
# Get Snowflake Session object
session = get_active_session()
session.sql_simplifier_enabled = True

# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"e2e_ml_snowparkpython", 
                     "version":{"major":1, "minor":0,},
                     "attributes":{"is_quickstart":1}}

# Current Environment Details
print('Connection Established with the following parameters:')
print('User      : {}'.format(session.get_current_user()))
print('Role      : {}'.format(session.get_current_role()))
print('Database  : {}'.format(session.get_current_database()))
print('Schema    : {}'.format(session.get_current_schema()))
print('Warehouse : {}'.format(session.get_current_warehouse()))

### Use the Snowpark DataFrame Reader to read in data from the externally staged `diamonds` CSV file 

In setup.sql, we staged the `diamonds.csv` file from an external s3 bucket. Now, we can read it in.

For more information on loading data, see documentation on [snowflake.snowpark.DataFrameReader](https://docs.snowflake.com/ko/developer-guide/snowpark/reference/python/api/snowflake.snowpark.DataFrameReader.html).




In [ ]:
# Create a Snowpark DataFrame that is configured to load data from the CSV file
# We can now infer schema from CSV files.
diamonds_df = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@DIAMONDS_ASSETS")

diamonds_df

In [ ]:
# Look at descriptive stats on the DataFrame
diamonds_df.describe()

In [ ]:
diamonds_df.columns

In [ ]:
# Force headers to uppercase
for colname in diamonds_df.columns:
    if colname == '"table"':
       new_colname = "TABLE_PCT"
    else:
        new_colname = str.upper(colname)
    diamonds_df = diamonds_df.with_column_renamed(colname, new_colname)

diamonds_df

In [ ]:
llm = 'claude-3-5-sonnet'

prompt = f"""
I used Snowparks describe function to calculate count, mean, stddev, min and max per column.
I want to build a machine learning model using both numeric and categorical features to predict PRICE.
What feature engineering steps should I perform based on their statistics from describe()? 
Provide a short Python code that does it for me.

The dataframe:
{diamonds_df.describe().to_pandas().to_markdown()}
"""

response = complete(llm, prompt)
st.markdown(response)

In [ ]:
diamonds_pdf = diamonds_df.to_pandas()

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

def engineer_features(diamonds_pdf):
    # 1. Handle outliers in dimensions (X, Y, Z)
    dimension_cols = ['X', 'Y', 'Z']
    df = diamonds_pdf[~(diamonds_pdf[dimension_cols] == 0).any(axis=1)]  # Remove 0 dimensions
    
    # 2. Create interaction features
    df['VOLUME'] = df['X'] * df['Y'] * df['Z']
    df['RATIO'] = df['X'] / df['Y']  # length/width ratio
    
    # 3. Define feature groups
    numeric_features = ['CARAT', 'DEPTH', 'TABLE_PCT', 'X', 'Y', 'Z', 'VOLUME', 'RATIO']
    categorical_features = ['CUT', 'COLOR', 'CLARITY']
    
    # 4. Create ordinal mappings for categorical features
    cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
    color_order = ['J', 'I', 'H', 'G', 'F', 'E', 'D']  # D is best
    clarity_order = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']  # IF is best
    
    # 5. Create preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OrdinalEncoder(categories=[cut_order, color_order, clarity_order]), 
             categorical_features)
        ])
    
    # Store the PRICE column before transformation
    price = df['PRICE'].copy()
    
    # 6. Fit and transform the data
    df_transformed = preprocessor.fit_transform(df)
    
    # Create new dataframe with transformed features
    feature_names = (numeric_features + 
                    [f"{feat}_ENCODED" for feat in categorical_features])
    df_transformed = pd.DataFrame(
        df_transformed, 
        columns=feature_names
    )
    
    # Add back the PRICE column to the transformed dataframe
    df_transformed['PRICE'] = price.values
    
    return df_transformed, preprocessor

df_transformed, preprocessor = engineer_features(diamonds_pdf)

In [ ]:
session.write_pandas(df_transformed,auto_create_table=True,table_type='temp',table_name='DIAMONDS_PREP',overwrite=True)

In [ ]:
spdf = session.table('diamonds_prep')
spdf.show()

# Time to prepare for machine learning!

In [ ]:
corr_diamonds_df = correlation(df=spdf)
corr_diamonds_df # This is a Pandas DataFrame

In [ ]:
# # Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr_diamonds_df, dtype=bool))

# # Create a heatmap with the features
plt.figure(figsize=(7, 7))
heatmap = sns.heatmap(corr_diamonds_df, mask=mask, cmap="YlGnBu", annot=True, vmin=-1, vmax=1)


In [ ]:
# Set up a plot to look at CARAT and PRICE
counts = spdf.to_pandas().groupby(['PRICE', 'CARAT', 'CLARITY_ENCODED']).size().reset_index(name='Count')

fig, ax = plt.subplots(figsize=(20, 20))
plt.title('Price vs Carat', fontsize=28)
ax = sns.scatterplot(data=counts, x='CARAT', y='PRICE', size='Count', hue='CLARITY_ENCODED', markers='o')
ax.grid(axis='y')

# The relationship is not linear - it appears exponential which makes sense given the rarity of the large diamonds
sns.move_legend(ax, "upper left")
sns.despine(left=True, bottom=True)

In [ ]:
# Split the data into train and test sets
diamonds_train_df, diamonds_test_df = spdf.random_split(weights=[0.9, 0.1], seed=0)
diamonds_train_df

In [ ]:
INPUT_COLS = [col for col in spdf.columns if col != 'PRICE']
INPUT_COLS

In [ ]:
LABEL_COLUMN = ['PRICE']
LABEL_COLUMN

In [ ]:
OUTPUT_COLUMN = ['PREDICTED_PRICE']
OUTPUT_COLUMN

In [ ]:
# Define the XGBRegressor
regressor = XGBRegressor(
    input_cols=INPUT_COLS,
    label_cols=LABEL_COLUMN,
    output_cols=OUTPUT_COLUMN
)

# Train
regressor.fit(diamonds_train_df)

# Predict
result = regressor.predict(diamonds_test_df)
result

In [ ]:
mape = mean_absolute_percentage_error(df=result, 
                                        y_true_col_names="PRICE", 
                                        y_pred_col_names="PREDICTED_PRICE")
mape

In [ ]:
# Plot actual vs predicted 
g = sns.relplot(data=result["PRICE", "PREDICTED_PRICE"].to_pandas().astype("float64"), x="PRICE", y="PREDICTED_PRICE", kind="scatter")
g.ax.axline((0,0), slope=1, color="r")

plt.show()

## Now let's put this model in the Model Registry for model management and governance

In [ ]:
# Get sample input data to pass into the registry logging function
X = diamonds_train_df.select(INPUT_COLS).limit(100)

db = identifier._get_unescaped_name(session.get_current_database())
schema = identifier._get_unescaped_name(session.get_current_schema())

# Define model name
model_name = "DIAMONDS_PRICE_PREDICTION"

# Create a registry and log the model
native_registry = Registry(session=session, database_name=db, schema_name=schema)

# Let's first log the very first model we trained
model_ver = native_registry.log_model(
    model_name=model_name,
    model=regressor,
    sample_input_data=X, # to provide the feature schema
    options={"enable_explainability": True}
)

# Add evaluation metric
model_ver.set_metric(metric_name="mean_abs_pct_err", value=mape)

# Add a description
model_ver.comment = "This is the first iteration of our Diamonds Price Prediction model. It is used for demo purposes."

model_ver

#### List models in registry, then load a model version and make predictions

In [ ]:
native_registry.show_models()

In [ ]:
model = native_registry.get_model('DIAMONDS_PRICE_PREDICTION')
model

In [ ]:
model.show_versions()

In [ ]:
model_version = model.version("DEFAULT")
model_version

In [ ]:
model_version.show_functions()

In [ ]:
preds = model_version.run(diamonds_test_df, function_name='PREDICT')
preds

In [ ]:
preds_with_explanations = model_version.run(diamonds_test_df, function_name='EXPLAIN')
preds_with_explanations

In [ ]:
import shap

explanation_columns = [col for col in preds_with_explanations.columns if 'explanation' in col.lower()]
explanation_df = preds_with_explanations[explanation_columns].to_pandas()


shap_exp = shap._explanation.Explanation(explanation_df.values, feature_names = explanation_df.columns) # wrapping them into a SHAP recognized object
shap.plots.bar(shap_exp)